In [2]:
import tweepy
import json
import os
import time
import pprint
import requests
import random
import networkx as nx


In [3]:

path="data_proj/"
pp=pprint.PrettyPrinter()

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

In [4]:
api_key="x"
api_secret="x"

access_token="x"
access_secret="x"

bearer_token="x"

In [5]:
auth=tweepy.OAuthHandler(api_key,api_secret)
auth.set_access_token(access_token,access_secret)
api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print("Auth success")


Auth success


In [6]:
users=["mizzaro","damiano10","miccighel_","eglu81","KevinRoitero"]
f_ers=[156,785,331,540,103]
f_ing=[331,836,211,621,256]
utenti=[]
for i in range(0,5):
    utente={}
    utente["name"]=users[i]
    utente["followers"]=f_ers[i]
    utente["following"]=f_ing[i]
    utenti.append(utente)
pp.pprint(utenti)

[{'followers': 156, 'following': 331, 'name': 'mizzaro'},
 {'followers': 785, 'following': 836, 'name': 'damiano10'},
 {'followers': 331, 'following': 211, 'name': 'miccighel_'},
 {'followers': 540, 'following': 621, 'name': 'eglu81'},
 {'followers': 103, 'following': 256, 'name': 'KevinRoitero'}]


In [ ]:
#1. Scaricate
#codice da non eseguire a meno che non vogliate bruciarvi tutte le richieste di twitter API
followers,following={},{}

for utente in utenti:
    followers_utente,following_utente=[],[]
    for item in tweepy.Cursor(
        api.followers,
        screen_name=utente["name"],
        skip_status=True,
        include_user_entities=False
    ).items():
        json_data=item._json
        new_follower={}
        new_follower["id"]=json_data["id"]
        new_follower["name"]=json_data["name"]
        new_follower["screen_name"]=json_data["screen_name"]
        new_follower["location"]=json_data["location"]
        new_follower["followers_count"]=json_data["followers_count"]
        new_follower["friends_count"]=json_data["friends_count"]
        new_follower["statuses_count"]=json_data["statuses_count"]
        
    followers_utente.append(new_follower)


    followers[utente["name"]]=followers_utente

    for item in tweepy.Cursor(
        api.friends,
        screen_name=utente["name"],
        skip_status=True,
        include_user_entities=False
    ).items():
        json_data=item._json
        new_following={}
        new_following["id"]=json_data["id"]
        new_following["name"]=json_data["name"]
        new_following["screen_name"]=json_data["screen_name"]
        new_following["location"]=json_data["location"]
        new_following["followers_count"]=json_data["followers_count"]
        new_following["friends_count"]=json_data["friends_count"]
        new_following["statuses_count"]=json_data["statuses_count"]
    
    following_utente.append(new_following)

    following[utente["name"]]=following_utente


#export to json
serialize_json("data_proj","followers_5_utenti.json",followers)
serialize_json("data_proj","following_5_utenti.json",following)

In [7]:
followers=read_json("json_5_utenti/finale/followers.json")
following=read_json("json_5_utenti/finale/following.json")
#Temporaneamente ho unito i followers/ing di ogni utente in due file separati, non so se è giusto

Data read from path: json_5_utenti/finale/followers.json
Data read from path: json_5_utenti/finale/following.json


In [8]:
#2 
random_followers_utenti,followers_of_followers={},{}

for utente in followers:
    #followers={av1:[],av2:[],...}
    #utente="Iron_Man","thorofficial",...
    random_followers_utenti[utente]=random.sample(followers[utente],5)
#result: random_followers_utenti= {"mizzaro":[5 utenti {}],"damiano10":[5 utenti {}],...}

try:
    for utente in random_followers_utenti:
        for f in random_followers_utenti[utente]:
            fof=[]
            time.sleep(10)
            for item in tweepy.Cursor(
                api.followers,
                screen_name=f["screen_name"],
                skip_status=True,
                include_user_entities=False
            ).items(10):
                json_data=item._json
                user={}
                user["id"]=json_data["id"]
                user["name"]=json_data["name"]
                user["screen_name"]=json_data["screen_name"]
                user["location"]=json_data["location"]
                user["followers_count"]=json_data["followers_count"]
                user["friends_count"]=json_data["friends_count"]
                user["statuses_count"]=json_data["statuses_count"]
                
                fof.append(user)
            followers_of_followers[f["screen_name"]]=fof
except tweepy.TweepError as error: #Le prime volte mi ha tornato TweepError:Unauthorized, ma ora non più
    print(error)
serialize_json("json_5_utenti/random_fof","followers_of_followers.json",followers_of_followers)

ValueError: Sample larger than population or is negative

In [ ]:
#3

random_following_utenti,following_of_following={},{}

for utente in following:
    random_following_utenti[utente]=random.sample(following[utente],5)

try:
    for utente in random_following_utenti:
        for f in random_following_utenti[utente]:
            fof=[]
            time.sleep(10)
            for item in tweepy.Cursor(
                api.following,
                screen_name=f["screen_name"],
                skip_status=True,
                include_user_entities=False
            ).items(10):
                json_data=item._json
                user={}
                user["id"]=json_data["id"]
                user["name"]=json_data["name"]
                user["screen_name"]=json_data["screen_name"]
                user["location"]=json_data["location"]
                user["followers_count"]=json_data["followers_count"]
                user["friends_count"]=json_data["friends_count"]
                user["statuses_count"]=json_data["statuses_count"]

                fof.append(user)
            following_of_following[f["screen_name"]]=fof
except tweepy.TweepError as error: #Le prime volte mi ha tornato TweepError:Unauthorized, ma ora non più
    print(error)
serialize_json("json_5_utenti/random_fof","following_of_following.json",following_of_following)

In [1]:
#TODO: Unire tutti i json in un singolo json sanificato da ripetizioni:
    #followers.json
    #following.json
    #followers_of_followers.json
    #following_of_following.json
    #SINTASSI per unire dictionaries:
        #dict_finale={**d1, **d2, ...}

In [4]:
#5 Creazione del grafo
in_edges = 
out_edges = 

grafo=nx.DiGraph()

for user in followers: #works in th
    for f in followers[user]:
        if f["location"]="":
            f["location"]="undefined"
        grafo.add_node(f["id"],name=f["name"],screen_name=f["screen_name"],location=f["location"],)
       
        #i nodi vanno caricati creano un grafo orientato, quindi è da vedere, magari creare degli oggetti con le relative proprietà
    
#TODO: creare edges tra nodi(struttura del tipo if(nodoA compare in follows di nodoB)then{edge nodoB to nodoA}

In [ ]:
#6 grafo interattivo di base
from pyvis.network import Network
twitter_graph = nx.read_gpickle("data/twitter_graph.pkl")
nt = Network(
    height ="100%",
    width = "100%",
    bgcolor="#222222",
    font_color="white",
    heading="Twitter Graph"
)
nt.barnes_hut()
nt.from_nx(twitter_graph)
neighbor_map = nt.get_adj_list()
for node in nt.nodes:
    node["value"] = len(neighbor_map[node["id"]])
nt.show("twitter_graph.html")


In [ ]:
#7 Proprietà del grafo
if nx.is_connected(twitter_graph):
    print("Il grafo è connesso.")
else:
    print("Il grafo non è connesso.")
    
if nx.is_bipartite(twitter_graph):
    print("Il grafo è bipartito.")
else:
    print("Il grafo non è bipartito.")
#8 Distanze sul grafo
centro = nx.center(twitter_graph)
diametro = nx.diameter(twitter_graph)
raggio = nx.radius(twitter_graph)

#9 Misure di centralità sul grafo
btw_centrality = betweenness_centrality(twitter_graph, nodes)
cls_centralitu = closeness_centrality(twitter_graph, nodes)
dg_centrality = degree_centrality(twitter_graph, nodes)
in_centrality= in_degree_centrality(twitter_graph)
out_centrality= out_degree_centrality(twitter_graph)
pagerank(twitter_graph)
hits(twitter_graph)


#Sottografo del nodo damiano10
sub_graph_damiano10 = ego_graph(twitter_graph, "damiano10")
max_clique(sub_graph_damiano10)
large_clique_size(sub_graph_damiano10)


#11 copertua minima degli archi
min_edge_cover(twitter_graph)

#12 Small-world-ness
omega(twitter_graph)
sigma(twitter_graph)

#13 Correlazione di Pearson e di Kendall (?)
scipy.stats.pearsonr()
scipy.stats.kendalltau()